In [1]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# loading csv file

df = pd.read_csv(r'dataset_5secondWindow\dataset_5secondWindow.csv')
df.head()

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,U1


In [3]:
df = df.drop(['user'], axis=1)
df = df.drop(['activityrecognition#0'], axis=1)


In [6]:
# ordinal encoding on categorical column

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

ct = asarray(df['target'])

oe = OrdinalEncoder()

df['target'] = oe.fit_transform(ct.reshape(-1,1))

In [7]:
#x and y

x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [8]:
# Spliting the data

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)


In [9]:
# normalising data

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [15]:
# importing classifiers and metrics

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
import time

In [16]:
# training models

classifiers = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Skl GBM": GradientBoostingClassifier(n_estimators=100),
    "Logistic Regression": LogisticRegression(max_iter=500),
    "SVM": SVC(kernel='linear')
    
}

In [17]:
results = pd.DataFrame({'Model': [], 'Score': [], 'Time': []})

for model_name, model in classifiers.items():
    start_time = time.time()
    pipe = make_pipeline(sc, model)

    pipe.fit(x_train, y_train)

    total_time = time.time() - start_time

    results = results.append({"Model": model_name,
                                'Score': pipe.score(x_test, y_test) * 100,
                                "Time": total_time},
                                ignore_index=True)

results_ord = results.sort_values(by=['Score'], ascending=False, ignore_index=True)
results_ord.index += 1
results_ord.style.bar(subset=['Score'], vmin=0, vmax=100, color='#5fba7d')

,Model,Score,Time
1,Skl GBM,99.491094,35.805068
2,Random Forest,98.897371,2.121206
3,Decision Tree,97.794741,0.305006
4,SVM,92.451230,0.806078
5,Logistic Regression,89.058524,0.738045


In [ ]:
results = pd.DataFrame({'Model': [], 'Score_train': [], 'Score_test':[]})

for model_name, model in classifiers.items():

    model.fit(x_train, y_train)
    results = results.append({"Model": model_name,
                              'Score_train': model.score(sc.fit_transform(x_train), y_train) * 100,
                              'Score_test': model.score(sc.transform(x_test), y_test) * 100},
                              ignore_index=True)

results_ord = results.sort_values(by=['Score_train'], ascending=False, ignore_index=True)
results_ord.index += 1
results_ord.style.bar(subset=['Score_train', 'Score_test'], vmin=0, vmax=100, color='#5fba7d')